In [0]:
#Import files module
import pandas as pd
import csv
import re                     ##Regular expressions
import glob
import os
import logging
import numpy as np
import datetime
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from math import sqrt
from numpy import concatenate
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [0]:
import pandas as pd
import numpy as np

In [0]:
#logging mechanism for debugging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [0]:
from google.colab import files
uploaded = files.upload()


Saving Route251.csv to Route251.csv


In [0]:
data = pd.read_csv("Route251.csv")

In [0]:
data

,Timeslot,Trips,Passengers
0,4,2,34
1,5,5,152
2,6,6,231
3,7,5,441
4,8,6,567
...,...,...,...
611,21,5,389
612,22,4,256
613,23,5,113
614,0,2,79


In [0]:
#Making separate dataframes for each separate timeslot over many days
df1=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df2=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df3=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df4=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df5=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df6=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df7=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df8=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df9=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df10=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df11=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df12=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df13=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df14=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df15=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df16=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df17=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df18=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df19=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df20=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])
df21=pd.DataFrame(columns=['TimeSlot','Trips','Passengers'])

In [0]:
df=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20,df21]

In [0]:
Timeslot_values= ['4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23']
Days = {'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}

In [0]:
#Obtaining the day and date details for current day 
import datetime
now = datetime.datetime.now()   
current_day = now.day
current_dow = Days[now.strftime("%A")]           #Obtaining current day of the week

In [0]:
arr = data.Timeslot.unique()

In [0]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [0]:
rmse_total = 0
trips = []
for i in arr :
  df = data.loc[data['Timeslot'] == i]
  dataset = df.drop(['Timeslot','Passengers'],axis = 1)
  values = dataset.values
  # ensure all data is float
  values = values.astype('float32')
  # normalize features
  scaler = MinMaxScaler(feature_range=(0, 1))
  scaled = scaler.fit_transform(values)
  # frame as supervised learning
  reframed = series_to_supervised(scaled, 1, 1)
  values = reframed.values
  train_set = values[:, :]
  test_set = values[:, :]
  # split into input and outputs
  train_X, train_y = train_set[:-1,:-1], train_set[:-1,-1]
  test_X, test_y = test_set[-1:,:-1], test_set[-1:,-1]
    
  # reshape input to be 3D [samples, timesteps, features]
  train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
  test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
    
  # design network
  model = Sequential()
  model.add(LSTM(5, input_shape=(train_X.shape[1], train_X.shape[2])))    #Adding LSTM units into the RNN
  model.add(Dense(1))
  model.compile(loss='mae', optimizer='adam')                             #mae-Mean Absolute error, Adam optimization rule
  # fit network
  history = model.fit(train_X, train_y, epochs=20, batch_size=3, validation_data=(test_X, test_y), verbose=1, shuffle=False)
  #obtaining predictions
  test_prediction = model.predict(test_X)
  train_prediction = model.predict(train_X)
  test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
  train_X = train_X.reshape((train_X.shape[0], train_X.shape[2]))

    # invert scaling for forecast
  print (test_prediction)
  actual_prediction = concatenate((test_prediction, test_X[:, 1:]), axis=1)
  actual_prediction = scaler.inverse_transform(actual_prediction)
  actual_prediction = actual_prediction[:,0]
  for x in actual_prediction:
      print (round(x))
      if round(x)<=0:
        trips.append(0)
      else:
        trips.append(round(x))
  actual_train_prediction = concatenate((train_prediction, train_X[:, 1:]), axis=1)
  actual_train_prediction = scaler.inverse_transform(actual_train_prediction)
  actual_train_prediction = actual_train_prediction[:,0]
    
  #invert scaling for actual ie. already available predictions(teacher signal)
  test_y = test_y.reshape((len(test_y), 1))
  inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
  inv_y = scaler.inverse_transform(inv_y)
  inv_y = inv_y[:,0]
  print (inv_y)
  #print df[i]['Trips']
    
  #invert scaling for actual ie. already available predictions(teacher signal) for training set
  train_y = train_y.reshape((len(train_y),1))
  inv_y_train = concatenate((train_y, train_X[:, 1:]), axis=1)
  inv_y_train = scaler.inverse_transform(inv_y_train)
  inv_y_train = inv_y_train[:,0]

Train on 26 samples, validate on 1 samples
Epoch 1/20
26/26 [==============================] - 1s 24ms/step - loss: 0.5059 - val_loss: 0.0241
Epoch 2/20
26/26 [==============================] - 0s 882us/step - loss: 0.4962 - val_loss: 0.0370
Epoch 3/20
26/26 [==============================] - 0s 812us/step - loss: 0.4860 - val_loss: 0.0504
Epoch 4/20
26/26 [==============================] - 0s 813us/step - loss: 0.4755 - val_loss: 0.0643
Epoch 5/20
26/26 [==============================] - 0s 783us/step - loss: 0.4645 - val_loss: 0.0788
Epoch 6/20
26/26 [==============================] - 0s 838us/step - loss: 0.4531 - val_loss: 0.0940
Epoch 7/20
26/26 [==============================] - 0s 855us/step - loss: 0.4411 - val_loss: 0.1099
Epoch 8/20
26/26 [==============================] - 0s 745us/step - loss: 0.4286 - val_loss: 0.1266
Epoch 9/20
26/26 [==============================] - 0s 810us/step - loss: 0.4154 - val_loss: 0.1441
Epoch 10/20
26/26 [==============================] - 0s 75

In [0]:
print (trips)
    

[1.0, 4.0, 5.0, 4.0, 5.0, 6.0, 6.0, 6.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 2.0, 1.0]
